<a href="https://colab.research.google.com/github/anushka-code/Feature-Envy-Code-Smell-Classification/blob/main/Feature_Envy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Left Branch of the NN-Architecture : CNN for Structural Object Oriented Metrics Feature Extraction





###Mounting Google Drive

In [2]:
!pip install -U -q PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
    
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
import numpy as np
import pandas as pd

Long Parameters Dataset

In [5]:
link = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('long_parameters_list_structural.csv')  
  
df_lp = pd.read_csv('long_parameters_list_structural.csv')
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,0,6,1,0,2,3,0,0,0,0.0,0,7,2,16,?,2,3,9,12,?,4,221,172,406,0.177778,1,9,34,1,29,0.961111,?,18,1.812500,2.000000,6,9,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,0,1,1,0,0,1,0,0,0,0.0,1,0,0,3,1,0,0,3,1,0,1,16,16,0,1.000000,0,0,3,0,3,0.500000,1,3,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,0,1,0,1,3,0,0,1,0,1.0,1,29,18,123,17,50,138,94,108,77,2,1957,1669,51801,0.012613,87,145,571,0,273,0.992107,0.404145078,234,2.219512,2.489362,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1,21,0,92,?,0,0,71,5,?,?,288,206,0,0.000000,5,15,92,0,92,1.010989,0,71,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,0,2,0,1,0,0,0,1,0,1.0,1,0,0,3,0,1,1,3,0,0,1,12,12,0,0.000000,0,1,4,0,3,0.000000,1,3,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Dataset

In [20]:
link= 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('switch_statements_structural.csv')  
  
df_ss = pd.read_csv('switch_statements_structural.csv')
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_switch_statements
0,1,2,0,0,2,1,5,1,0,1,0,1,0,1,4,0,0,2,0,0.5,0,2,0,7,?,1,3,5,2,?,3,53,39,55,0.100000,0,4,13,2,8,0.916667,?,6,1.142857,1.200000,7,47,3,439,53,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,2,6,3,0,0,0,1,1,0,1,1,0,0,0,0.0,0,8,0,30,12,4,10,22,8,111,3,377,310,1214,0.057143,0,32,208,0,61,0.943966,0.681818182,51,2.033333,2.318182,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,2,0,0,0,0,1,3,1,0,2,0,1,0,1,2,0,0,1,0,1.0,1,0,0,1,0,0,1,1,0,0,1,3,3,4,1.000000,0,3,2,?,1,0.000000,?,1,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,0,0,0,0,2,3,1,0,0,0,0,0,1,2,0,0,1,0,1.0,1,0,0,1,0,0,1,1,0,0,1,5,5,4,1.000000,0,1,2,?,1,0.000000,?,1,1.000000,1.000000,18,102,0,1212,120,43,849,78,5788,4778,65687,False
4,0,0,0,0,0,1,3,1,0,0,0,6,0,0,0,0,0,0,0,0.0,0,2,1,16,0,7,10,14,6,8,3,197,179,640,0.305556,0,10,41,0,31,0.855556,1,28,1.937500,2.000000,4,27,0,516,29,43,849,78,5788,4778,65687,False


### Data Pre-Processing of the Code Smell Datasets

In [15]:
import numpy as np
df_lp.rename(columns = {'is_long_parameters_list':'is_code_smell'}, inplace = True)
df_lp["is_code_smell"] = df_lp["is_code_smell"].astype(int)
Y_lp = df_lp.iloc[:,-1:]
X_lp = df_lp.iloc[:,:82]
X_lp = X_lp.replace(to_replace =["?"], value = np.nan)
X_lp = X_lp.astype(float)

In [25]:
X_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_code_smell
0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,2.0,0.0,0.0,0.0,6.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,16.0,NaN,2.0,3.0,9.0,12.0,NaN,4.0,221.0,172.0,406.0,0.177778,1.0,9.0,34.0,1.0,29.0,0.961111,NaN,18.0,1.812500,2.000000,6.0,9.0,0.0,229.0,16.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,16.0,16.0,0.0,1.000000,0.0,0.0,3.0,0.0,3.0,0.500000,1.000000,3.0,1.000000,1.000000,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,29.0,18.0,123.0,17.0,50.0,138.0,94.0,108.0,77.0,2.0,1957.0,1669.0,51801.0,0.012613,87.0,145.0,571.0,0.0,273.0,0.992107,0.404145,234.0,2.219512,2.489362,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
3,0.0,5.0,0.0,0.0,9.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0,0.0,92.0,NaN,0.0,0.0,71.0,5.0,NaN,NaN,288.0,206.0,0.0,0.000000,5.0,15.0,92.0,0.0,92.0,1.010989,0.000000,71.0,1.000000,1.000000,34.0,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
4,1.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,3.0,0.0,0.0,1.0,12.0,12.0,0.0,0.000000,0.0,1.0,4.0,0.0,3.0,0.000000,1.000000,3.0,1.000000,1.000000,34.0,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0


In [26]:
Y_lp.head(5)

,is_code_smell
0,0
1,0
2,0
3,0
4,0


In [29]:
X_lp.shape

(420, 57)

In [30]:
Y_lp.shape

(420, 1)

In [21]:
import numpy as np
df_ss.rename(columns = {'is_switch_statements':'is_code_smell'}, inplace = True)
df_ss["is_code_smell"] = df_ss["is_code_smell"].astype(int)
Y_ss = df_ss.iloc[:,-1:]
X_ss = df_ss.iloc[:,:82]
X_ss = X_ss.replace(to_replace =["?"], value = np.nan)
X_ss = X_ss.astype(float)

In [23]:
X_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_code_smell
0,1.0,2.0,0.0,0.0,2.0,1.0,5.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,4.0,0.0,0.0,2.0,0.0,0.5,0.0,2.0,0.0,7.0,NaN,1.0,3.0,5.0,2.0,NaN,3.0,53.0,39.0,55.0,0.100000,0.0,4.0,13.0,2.0,8.0,0.916667,NaN,6.0,1.142857,1.200000,7.0,47.0,3.0,439.0,53.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
1,0.0,0.0,0.0,0.0,0.0,2.0,6.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,30.0,12.0,4.0,10.0,22.0,8.0,111.0,3.0,377.0,310.0,1214.0,0.057143,0.0,32.0,208.0,0.0,61.0,0.943966,0.681818,51.0,2.033333,2.318182,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
2,2.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,3.0,3.0,4.0,1.000000,0.0,3.0,2.0,NaN,1.0,0.000000,NaN,1.0,1.000000,1.000000,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
3,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0,1.000000,0.0,1.0,2.0,NaN,1.0,0.000000,NaN,1.0,1.000000,1.000000,18.0,102.0,0.0,1212.0,120.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,16.0,0.0,7.0,10.0,14.0,6.0,8.0,3.0,197.0,179.0,640.0,0.305556,0.0,10.0,41.0,0.0,31.0,0.855556,1.000000,28.0,1.937500,2.000000,4.0,27.0,0.0,516.0,29.0,43.0,849.0,78.0,5788.0,4778.0,65687.0,0.0


In [24]:
Y_ss.head(5)

,is_code_smell
0,0
1,0
2,0
3,0
4,0


In [32]:
X_ss.shape

(420, 57)

In [33]:
Y_ss.shape

(420, 1)

### Concatenating the Datasets

In [35]:
import pandas as pd
code_smells = [X_lp, X_ss]
X_train = pd.concat(code_smells)
type(X_train)

pandas.core.frame.DataFrame

In [37]:
X_train.shape

(840, 57)

In [38]:
import pandas as pd
labels = [Y_lp, Y_ss]
Y_train = pd.concat(labels)
type(Y_train)

pandas.core.frame.DataFrame

In [39]:
Y_train.shape

(840, 1)

In [51]:
Y_train['is_code_smell'].value_counts()

0    573
1    267
Name: is_code_smell, dtype: int64

###SMOTE's Algorithm - For Oversampling


###Data Normalization

In [47]:
from sklearn.preprocessing import MinMaxScaler
# compute required values
scaler = MinMaxScaler()
model = scaler.fit(X_train)
X_train = model.transform(X_train)
print(X_train)
X_train.shape

[[0.         0.         0.         ... 0.3325772  0.19318279 0.        ]
 [0.         0.         0.         ... 0.3325772  0.19318279 0.        ]
 [0.0625     0.         0.         ... 0.3325772  0.19318279 0.        ]
 ...
 [0.0625     0.         0.09638554 ... 0.62322552 1.         1.        ]
 [0.0625     0.         0.06024096 ... 0.62322552 1.         1.        ]
 [0.125      0.15789474 0.13253012 ... 0.62322552 1.         1.        ]]


(840, 57)

In [49]:
X_train_final = X_train.reshape((840,57,1))

### 1-D CNN Implementation

In [52]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import np_utils

In [54]:
model = Sequential()
model.add(Convolution1D(filters=128, kernel_size=3, activation='relu', input_shape=(57,1)))
model.add(Convolution1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#check pretrained models for feature extraction of CNN 2d data 
#which is good for numerical data
#modelfit xd boost (better version)
#imbalanced data - synthetic data addition 
#random forest

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 55, 128)           512       
                                                                 
 conv1d_4 (Conv1D)           (None, 53, 128)           49280     
                                                                 
 dropout_1 (Dropout)         (None, 53, 128)           0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 26, 128)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 22, 128)           82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 11, 128)          0         
 1D)                                                  

In [58]:
hist = model.fit(X_train_final,Y_train,epochs=20,shuffle=True,batch_size=50,validation_split=0.10) 

Epoch 1/20
16/16 [==============================] - 1s 51ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 2/20
16/16 [==============================] - 1s 48ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 3/20
16/16 [==============================] - 1s 47ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 4/20
16/16 [==============================] - 1s 48ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 5/20
16/16 [==============================] - 1s 49ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 6/20
16/16 [==============================] - 1s 48ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 7/20
16/16 [==============================] - 1s 48ms/step - loss: nan - accuracy: 0.6839 - val_loss: nan - val_accuracy: 0.6667
Epoch 8/20
16/16 [==============================] - 1s 

###Forming Groups of Features 

Make one-hot encoded data for project, package, class and method groups

*   0 : method
*   1 : class
*   2 : package
*   3 : project





In [59]:
group_of_features = {
    "NOP_method" : 0,
    "CC_method" : 0,
    "ATFD_method" : 0,
    "FDP_method" : 0,
    "CM_method" : 0,
    "MAXNESTING_method" : 0,
    "LOC_method" : 0,
    "CYCLO_method" : 0,
    "NMCS_method" : 0,
    "NOLV_method" : 0,
    "MaMCL_method" : 0,
    "NOAV_method" : 0,
    "LAA_method" : 0,
    "FANOUT_method" : 0,
    "CFNAMM_method" : 0,
    "ATLD_method" : 0,
    "CLNAMM_method" : 0,
    "CINT_method" : 0,
    "MeMCL_method" : 0,
    "CDISP_method" : 0,
    "NOII_type" : 1,
    "NOAM_type" : 1,
    "NOCS_type" : 2,
    "NOM_type" : 1,
    "NMO_type" : 1,
    "ATFD_type" : 0,
    "FANOUT_type" : 1,
    "NOMNAMM_type" : 1,
    "NOA_type" : 1,
    "NIM_type" : 1,
    "DIT_type" : 1,
    "LOC_type" : 1,
    "LOCNAMM_type" : 1,
    "CFNAMM_type" : 1,
    "TCC_type" : 0,
    "NOPA_type" : 1,
    "CBO_type" : 1,
    "RFC_type" : 1,
    "NOC_type" : 1,
    "WMC_type" : 1,
    "LCOM5_type" : 0,
    "WOC_type" : 0,
    "WMCNAMM_type" : 1,
    "AMW_type" : 1,
    "AMWNAMM_type" : 1,
    "NOCS_package" : 2,
    "NOMNAMM_package" : 2,
    "NOI_package" : 2,
    "LOC_package" : 2,
    "NOM_package" : 2,
    "NOPK_project" : 3,
    "NOCS_project" : 3,
    "NOI_project" : 3,
    "NOM_project" : 3,
    "NOMNAMM_project" : 3,
    "LOC_project" : 3,
    "isStatic_type" : 1,
    "number_private_visibility_attributes" : 1,
    "number_protected_visibility_attributes" : 1,
    "number_package_visibility_attributes" : 1,
    "num_final_attributes" : 1,
    "num_static_attributes" : 1,
    "num_final_static_attributes" : 1,
    "num_not_final_not_static_attributes" : 1,
    "num_final_not_static_attributes" : 1,
    "num_static_not_final_attributes" : 1,
    "number_public_visibility_methods" : 0,
    "number_private_visibility_methods" : 0,
    "number_protected_visibility_methods" : 0,
    "number_package_visibility_methods" : 0,
    "number_final_methods" : 0,
    "number_abstract_methods" : 0,
    "number_not_abstract_not_final_methods" : 0,
    "number_static_methods" : 0,
    "number_final_static_methods" : 0,
    "number_final_not_static_methods" : 0,
    "number_not_final_static_methods" : 0,
    "number_not_final_not_static_methods" : 0,
    "number_standard_design_methods" : 0,
    "number_constructor_DefaultConstructor_methods" : 0,
    "number_constructor_NotDefaultConstructor_methods" : 0,
    "isStatic_method" : 0
}

print(len(group_of_features.keys()))

82


In [60]:
from collections import Counter
res = Counter(group_of_features.values())
print(res)

#hence we have methods = 40, class = 30, project = 6, package = 6

Counter({0: 40, 1: 30, 2: 6, 3: 6})


In [61]:
from keras.utils import np_utils
group_type = np.fromiter(group_of_features.values(), dtype=float)
group_type_one_hot = np_utils.to_categorical(group_type, dtype=float)
print(group_type_one_hot)
print(group_type_one_hot.shape)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0.

###Data Reshaping

In [ ]:
c = np.array([1,2,3,4])
x = np.tile(c,(4,1))
print(x)
x.shape

In [ ]:
group_type_one_hot_reshaped = group_type_one_hot.reshape(1,82,4)

In [ ]:
sample = np.tile(group_type_one_hot_reshaped,(715,1,1))
print(sample)
sample.shape

In [ ]:
new = X_train.reshape((715,82,1))
new.shape

In [ ]:
X_train_inter = np.dstack((new, sample))
X_train_inter.shape

In [ ]:
model = Sequential()
model.add(Convolution1D(filters=128, kernel_size=3, activation='relu', input_shape=(82,5)))
model.add(Convolution1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(X_train_inter,Y,epochs=50,shuffle=True,batch_size=100,validation_split=0.20) 